In [1]:
import pandas as pd
import numpy as np


chrom = 22
df = pd.read_pickle('chrom_%d.pkl' % chrom)
df.set_index(['gene', 'intron'], inplace=True)
df.sort_index(inplace=True)
df.head()

chrom  lmm-pval  lmm-rank-pval       pos  qep-pval  \
gene            intron                                                       
ENSG00000015475 7          22  0.190543       0.083220  18207220  0.235661   
                7          22  0.990685       0.813105  18207251  0.639112   
                7          22  0.367290       0.777673  18207730  0.867640   
                7          22  0.524754       0.607386  18207780  0.418764   
                7          22  0.836786       0.175274  18207927  0.758660   

                                      snp_id  
gene            intron                        
ENSG00000015475 7            snp_22_18207220  
                7            snp_22_18207251  
                7            snp_22_18207730  
                7       indel:1I_22_18207780  
                7            snp_22_18207927

# Bonferroni correction

In [2]:
ntests = len(df)
print("Number of tests: %d" % ntests)
df['lmm-pval'] = np.clip(df['lmm-pval'] * ntests, 0, 1)
df['lmm-rank-pval'] = np.clip(df['lmm-rank-pval'] * ntests, 0, 1)
df['qep-pval'] = np.clip(df['qep-pval'] * ntests, 0, 1)

Number of tests: 145721


In [3]:
from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar
output_notebook()

Loading BokehJS ...

In [4]:
def plot_for_thr(pval_thr, perc=False):

    data = pd.DataFrame(columns=('pval', 'method', 'color'))

    def append_df(data, df, name):
        N = len(df['%s-pval' % name])
        d = [np.asarray(df['%s-pval' % name]), [name] * N]
        data = pd.concat([data, pd.DataFrame({'pval': d[0], 'method': d[1], 'color': ['white'] * N})])
        return data

    data = append_df(data, df, 'lmm')
    data = append_df(data, df, 'lmm-rank')
    data = append_df(data, df, 'qep')

    data.loc[data['method'] == 'lmm', 'color'] = 'blue'
    data.loc[data['method'] == 'lmm-rank', 'color'] = 'green'
    data.loc[data['method'] == 'qep', 'color'] = 'red'


    if perc:
        grouped = data.groupby(data['method']).agg({'method': lambda x: x[0], 'pval': lambda x: 100*sum(x<=pval_thr)/float(len(x))})
    else:
        grouped = data.groupby(data['method']).agg({'method': lambda x: x[0], 'pval': lambda x: sum(x<=pval_thr)})

    grouped.loc['lmm', 'color'] = 'blue'
    grouped.loc['lmm-rank', 'color'] = 'green'
    grouped.loc['qep', 'color'] = 'red'

    p = Bar(grouped, values='pval', color='color', label='method', legend=False,
           title = "Chromossome %d :: p-value < %.0e" % (chrom, pval_thr), tools=['save,reset'], height=400, width=400)

    if perc:
        p.yaxis.axis_label = 'Percentage of hits (p-value < %.0e)' % (pval_thr, )
    else:
        p.yaxis.axis_label = 'Number of hits (p-value < %.0e)' % (pval_thr, )

    show(p)

# Number of Hits

In [5]:
plot_for_thr(1e-5)
plot_for_thr(1e-1)
plot_for_thr(1.0)

# Percentage of Hits

In [6]:
plot_for_thr(1e-5, True)
plot_for_thr(1e-1, True)
plot_for_thr(1.0, True)